In [1]:
!mkdir predicted_frames
!mkdir save_frames

#Run the snippets above first

mkdir: cannot create directory ‘predicted_frames’: File exists
mkdir: cannot create directory ‘save_frames’: File exists


In [2]:
# importing the necessary libraries
import cv2
import numpy as np
import uuid
import os

# Creating a VideoCapture object to read the video
cap = cv2.VideoCapture('data/test_video.mp4')

count=0
while (cap.isOpened()):

    ret, frame = cap.read()
    
    filename=str(uuid.uuid4())+".jpg"
    fullpath=os.path.join("./save_frames",filename)
    cv2.imwrite(fullpath, frame)
    count+=1
    
    if count==20:
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
!ls

data	   predicted_frames  save_frames	       video_processing.ipynb
image.png  README.md	     Solution_Methodology.pdf
lib	   requirements.txt  src


In [4]:
import tensorflow as tf
print(tf.__version__)

2024-11-22 12:08:23.054929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732248503.068910  118160 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732248503.073441  118160 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 12:08:23.088317: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.18.0


In [5]:
predicted_path = './predicted_frames'

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import zipfile
import cv2
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import uuid
import time

from object_detection.utils import label_map_util

KPIs_dict = dict()

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = "data/frozen_inference_graph.pb"

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = "data/labels.txt"

NUM_CLASSES = 7

# ## Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# ## Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# ## Helper code
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def calculate_area(x1, y1, x2, y2):
    xDiff = abs(x1 - x2)
    yDiff = abs(y1 - y2)
    area = xDiff * yDiff
    return area

def draw_bounding_boxes(image, boxes, classes, scores, category_index, min_score_thresh=0.5):
    for i in range(len(boxes)):
        if scores[i] > min_score_thresh:
            box = boxes[i]
            class_id = classes[i]
            score = scores[i]
            label = category_index[class_id]['name']
            
            ymin, xmin, ymax, xmax = box
            (left, right, top, bottom) = (int(xmin * image.shape[1]), int(xmax * image.shape[1]),
                                          int(ymin * image.shape[0]), int(ymax * image.shape[0]))
            
            # Draw bounding box
            cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # Draw label and score
            label_text = f"{label}: {int(score * 100)}%"
            cv2.putText(image, label_text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return image

# Size, in inches, of the output images.
IMAGE_SIZE = (500, 500)
count = 0
frame_number = 0
cap = cv2.VideoCapture('data/test_video.mp4')

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while cap.isOpened():
            frame_number += 1

            ret, frame = cap.read()

            filename = str(uuid.uuid4()) + ".jpg"
            fullpath = os.path.join("./save_frames", filename)
            cv2.imwrite(fullpath, frame)
            count += 1

            # for testing script...
            if count == 50:
                break
            # ends here

            image = Image.open(fullpath)

            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.

            image_np = draw_bounding_boxes(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                min_score_thresh=0.5)

            image_pil = Image.fromarray(np.uint8(image_np)).convert('RGB')
            im_width, im_height = image_pil.size
            area_whole = im_width * im_height
            num_detections = int(num_detections[0])  # Extract the single element from the array
            for i in range(num_detections):
                if scores[0][i] > 0.5:
                    ymin, xmin, ymax, xmax = boxes[0][i]
                    (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
                    area = calculate_area(top, left, bottom, right)

                    percent_area = round(float(area) / area_whole, 2)  # Convert to regular float
                    class_id = int(classes[0][i])
                    label = category_index[class_id]['name']

                    if label in KPIs_dict.keys():
                        KPIs_dict[label]['count'] += 1
                        KPIs_dict[label]['area'].append(percent_area)
                        KPIs_dict[label]['frames'].append(frame_number)
                    else:
                        KPIs_dict[label] = {"count": 1}
                        KPIs_dict[label].update({"area": [percent_area]})
                        KPIs_dict[label].update({"frames": [frame_number]})

            full_predicted_path = os.path.join(predicted_path, str(uuid.uuid4()) + ".jpg")
            cv2.imwrite(full_predicted_path, image_np)_path, image_np)

I0000 00:00:1732248930.544020  118160 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6926 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1


In [12]:
KPIs_dict

{'hotstar': {'count': 9,
  'area': [np.float32(0.16),
   np.float32(0.16),
   np.float32(0.21),
   np.float32(0.18),
   np.float32(0.15),
   np.float32(0.34),
   np.float32(0.31),
   np.float32(0.17),
   np.float32(0.16)],
  'frames': [4, 5, 5, 6, 10, 14, 16, 31, 34]},
 'pepsi': {'count': 2,
  'area': [np.float32(0.0), np.float32(0.0)],
  'frames': [46, 46]}}

In [15]:
def shortest_longest_area(area_list):
    area_list.sort()
    shortest = area_list[0]
    longest = area_list[-1]
    response = {
        "shortest":shortest,
        "longest":longest
    }
    return response

In [16]:
for each_brand,analytics_dict in KPIs_dict.items():
    area=analytics_dict['area']
    response = shortest_longest_area(area)
    KPIs_dict[each_brand].update(response)

In [17]:
KPIs_dict

{'hotstar': {'count': 9,
  'area': [np.float32(0.15),
   np.float32(0.16),
   np.float32(0.16),
   np.float32(0.16),
   np.float32(0.17),
   np.float32(0.18),
   np.float32(0.21),
   np.float32(0.31),
   np.float32(0.34)],
  'frames': [4, 5, 5, 6, 10, 14, 16, 31, 34],
  'shortest_area': np.float32(0.15),
  'longest_area': np.float32(0.34),
  'shortest': np.float32(0.15),
  'longest': np.float32(0.34)},
 'pepsi': {'count': 2,
  'area': [np.float32(0.0), np.float32(0.0)],
  'frames': [46, 46],
  'shortest_area': np.float32(0.0),
  'longest_area': np.float32(0.0),
  'shortest': np.float32(0.0),
  'longest': np.float32(0.0)}}